<a href="https://colab.research.google.com/github/Yoshiki0418/Deep_Learning/blob/main/DL_Lecture2/transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## transferlearning

In [1]:
# googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Deep_Learning/DL_Lecture2

/content/drive/MyDrive/Deep_Learning/DL_Lecture2


In [3]:
import torch
from torch.nn import functional as F
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch import optim, nn

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

%load_ext autoreload
%autoreload 2
import utils

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#GPUの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# 事前学習されたモデルと重みを読み込む
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 127MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

事前に学習されているモデルでは、最終層の`out_features`より、1000クラス分類になっていることがわかる。今回のタスクでは、10クラス分類にしたいので、最終層の全結合層部分に少し変更を加える必要がある。

In [6]:
# モデルの最終層を変更して10クラス分類になるようにする
num_classes = 10
model.fc = nn.Linear(512, num_classes)

In [8]:
# 最終層以外のパラメータは事前に学習された値で固定して転移学習を行う
for name, param in model.named_parameters():
    if not name.startswith('fc'):
        param.requires_grad = False

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./cifar10_data', train=True, download=True, transform=transform)
val_dataset = CIFAR10(root='./cifar10_data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=4)

# モデルをGPUに移動
model = model.to(device)

opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# resnetは大きいのでCUDAが必要
train_losses, val_losses, val_accuracies = utils.learn(model, train_loader, val_loader, opt, F.cross_entropy, 5)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                                                           

epoch: 0: train error: 0.7466224663111628, validation error: 0.7497190296649933, validation accuracy: 0.7675083696842193


epoch: 1: train error: 0.7317138229097638, validation error: 0.7366730272769928, validation accuracy: 0.7703025341033936


epoch: 2: train error: 0.71889535991513, validation error: 0.7245550096035004, validation accuracy: 0.7714305639266967


epoch: 3: train error: 0.7074714461151435, validation error: 0.7144618093967438, validation accuracy: 0.7728495657444


epoch: 4: train error: 0.6979904150476262, validation error: 0.7049749672412873, validation accuracy: 0.7740214407444
